In [10]:
# !pip freeze > requirements.txt

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.document_loaders import DirectoryLoader, UnstructuredHTMLLoader, UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter, SpacyTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [2]:
import os
import openai
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

os.environ["OPENAI_API_KEY"] = open_file('openai_api_key.txt')
openai.api_key = open_file('openai_api_key.txt')
openai_api_key = openai.api_key

In [5]:
def inputdocs():
    docs = os.listdir('Input')
    docs
    from bs4 import BeautifulSoup
    texts = []
    embeddings = OpenAIEmbeddings()
    for file in docs:
        try:
            with open("Input/"+file, "r") as f:
                html = f.read()
            soup = BeautifulSoup(html, "html.parser")
            print(soup.text)
            text_splitter = CharacterTextSplitter(chunk_size=1000, separator=" ")
            texts.extend(text_splitter.split_text(soup.text))
        except Exception as e:
            print(e)
            print("This file isn't supported")
    docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

This post is the encapsulation of my thoughts on AGI evolution, which I’ve variously written about eg here and here. Feel free to just check out the equation and skip to the last section if you’d rather cut to the chase and start the debate.There is a persistent worry amongst smart folks about the creation of smart and capable artificial intelligences, or AGI. The worry is that by its very nature it would have a very different set of moral codes (things it should or shouldn’t do) and a vastly increased set of capabilities, which can result in pretty catastrophic outcomes. Like a baby with a bazooka, but worse. Metaculus now predicts weak AGI by 2027 & strong AGI with robotic capabilities by 2038.To put my bias upfront, I’ve not found this to be very persuasive. From an epistemological point of view, “it can be dangerous” seems like insufficient grounds to worry, and “we should make it safer” seems like a non sequitur. Like sure, make it safer, but since nobody knows how it feels like a

In [13]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
def answerfromindex(query):
    query = "What is the Strange equation?" #input("What are you curious about?")
    docs = docsearch.similarity_search(query)
    template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    ALWAYS return a "SOURCES" part in your answer.

    QUESTION: {query}"""
    # =========
    # {summaries}
    # =========
    # FINAL ANSWER:"""
    # PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])
    chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine", return_intermediate_steps=True)#, prompt=PROMPT)
    chain({"input_documents": docs, "question": template}, return_only_outputs=True)

{'intermediate_steps': ['\nThe answer to the query is that Frank Drake created an equation to estimate the number of extra-terrestrial civilisations that could exist. This equation is known as the Drake equation and it takes into account the rate of star formation, fraction of those with planets, fraction which can support life and actually develop it, fraction of those with civilisations which will eventually become visible to us. The equation is used to get a sense of the inputs into the equations and to estimate the number of detectable civilizations in our galaxy.\n\nSOURCES:\n1. Drake, F. (1961). "Project Ozma and SETI". In Cocconi, G.; Morrison, P. (eds.). Searching for Interstellar Communications. New York: Benjamin. pp. 3–18.\n2. Drake, F. (1961). "Project Ozma and SETI". In Cocconi, G.; Morrison, P. (eds.). Searching for Interstellar Communications. New York: Benjamin. pp. 3–18.\n3. Drake, F. (1961). "Project Ozma and SETI". In Cocconi, G.; Morrison, P. (eds.). Searching for I

In [15]:
# Langchain ReAct framework
import os
import openai
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

os.environ["OPENAI_API_KEY"] = open_file('openai_api_key.txt')
openai.api_key = open_file('openai_api_key.txt')
openai_api_key = openai.api_key

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.utilities import BashProcess, GoogleSearchAPIWrapper
search = GoogleSearchAPIWrapper()
search.run("Obama's first name?")
bash = BashProcess()
with open("wolfram_key.txt", "r") as f:
    wolfram_alpha_appid = f.read().strip()
os.environ["GOOGLE_CSE_ID"] = open_file('google_searchengine_id.txt')
os.environ["GOOGLE_API_KEY"] = open_file('google_api_key.txt')

llm = OpenAI(temperature=0) # to set the llm
tool_names = ["python_repl", "wolfram-alpha","requests","terminal","podcast-api","wikipedia","searx-search","google-search"]

tools = load_tools(tool_names, llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("What is the comparative political economy of the Wookies compared to that of Tatooine?")


ValidationError: 1 validation error for GoogleSearchAPIWrapper
__root__
  Did not find google_api_key, please add an environment variable `GOOGLE_API_KEY` which contains it, or pass  `google_api_key` as a named parameter. (type=value_error)